In [1]:
import time
import glob
import pandas as pd

import requests

import pytz
import numpy as np
import networkx as nx

import mailbox
import email

import pylab
from pylab import cm
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

#from bigbang.analysis.listserv import ListservList
from tgpp.config.config import CONFIG

plt.style.use("publication_jcap.mplstyle")

In [10]:
box = mailbox.mbox('/Users/christovis/test.mbox', create=False)
mlist = []
for msg in list(box.values()):
    _msg = dict(msg)
    if msg.is_multipart():
        # if message has many submessages, only get the top one
        # TODO: it does not detect that
        # https://list.etsi.org/scripts/wa.exe?A2=3GPP_TSG_SA_WG3_LI;6b0e2163.2008B&S=
        # is multipart
        payloads = msg.get_payload()
        _msg["body"] = str(payloads[0].get_payload(decode=True))
    else:
        _msg["body"] = str(msg.get_payload(decode=True))
    mlist.append(_msg)
    
df = pd.DataFrame(mlist)

In [11]:
df[]

,Content-Transfer-Encoding,MIME-Version,sender,date,reply-to,subject,message-id,in-reply-to,Content-Type,from,Archived-At,Attachment-0,body
0,quoted-printable,1.0,"""3gpp_tsg_sa_wg3_li: (tsg sa wg3 lawful inte...","Sat, 26 Feb 2022 14:29:06 +0000",alex.leadbeater@BT.COM,'Re: Draft Agenda SA3#84-LI-e-b (Draft B)',<LO2P123MB4973CD3050D088FAA1A77F4D943F9@LO2P12...,<LO2P123MB49734D410320BFEB1AD8180B943D9@LO2P12...,"text/plain; charset=""utf-8""; MIME-Version=""1.0...",Alex Leadbeater <alex.leadbeater@BT.COM>,<https://list.etsi.org/scripts/wa.exe?A2=3GPP_...,3GPP TSG-SA3LI Meeting #84e-b S3i220102 draft ...,"b'Hi All,\n\nAttached is draft B adding a coup..."


In [16]:
import PyPDF2

def _read_pdf(data: str, source: str) -> str:
    text = []
    pdf = PyPDF2.PdfFileReader(data)
    for page_nr, page in enumerate(range(pdf.getNumPages())):
        try:
            text.append(pdf.getPage(page).extractText())
        except Exception:
            logger.info(f"For {source} page number {page_nr} couldn't be read")
            continue
    return (" ").join(text)


file_path = "/Users/christovis/Documents/InternetGovernance/proj1_3gpp_and_comp/standardization-of-lawful-interception-technologies-in-the-3GPP/data/reference_set/ts_133107v140200p.pdf"
text = _read_pdf(open(file_path, 'rb'), file_path)

KeyboardInterrupt: 

In [4]:
import yaml

filepath = "/home/christovis/InternetGov/bigbang/tests/data/address_header_test_file.yaml"
with open(filepath, "r") as stream:
    addresses = yaml.safe_load(stream)
    
addresses

{'"xuxiaodong@chinamobile.com" <xuxiaodong@CHINAMOBILE.COM>': "('xuxiaodong@chinamobile.com', 'xuxiaodong', 'chinamobile.com')",
 'Jacob John <jacobjohn@MOTOROLA.COM>': "('jacob john', 'jacobjohn', 'motorola.com')",
 '"Wuyuchun (Wu Yuchun, Hisilicon)" <wuyuchun@HUAWEI.COM>': "('wuyuchun (wu yuchun, hisilicon)', 'wuyuchun', 'huawei.com')",
 'xingjinqiang@chinamobile.com xingjinqiang@chinamobile.com': "(null, 'xingjinqiang', 'chinamobile.com')",
 'abdul rasheed m d rasheed@motorola.com': "('abdul rasheed m d', 'rasheed', 'motorola.com')",
 'fredrik =?utf-8?q?sundstr=c3=b6m?= fredrik.sundstrom@ericsson.com': "('fredrik =?utf-8?q?sundstr=c3=b6m?=', 'fredrik.sundstrom', 'ericsson.com')",
 'guozhili guozhili@starpointcomm.com': "('guozhili', 'guozhili', 'starpointcomm.com')",
 '=?utf-8?b?runjvc3lvkdnu6flrr4=?= zhangjibin@ecit.org.cn': "('=?utf-8?b?runjvc3lvkdnu6flrr4=?=', 'zhangjibin', 'ecit.org.cn')",
 'Xueming Pan <<a href="/scripts/wa.exe?LOGON=A2%3D3GPP_TSG_RAN%3B82086c35.2012B%26S%3D&am

In [41]:
for key, values in addresses.items():
    try:
        res = [s.lower() for s in list(values.values())]
        print(res)
    except:
        continue

['xuxiaodong@chinamobile.com', 'xuxiaodong', 'chinamobile.com']
['jacob john', 'jacobjohn', 'motorola.com']
['wuyuchun (wu yuchun, hisilicon)', 'wuyuchun', 'huawei.com']
['abdul rasheed m d', 'rasheed', 'motorola.com']
['fredrik =?utf-8?q?sundstr=c3=b6m?=', 'fredrik.sundstrom', 'ericsson.com']
['guozhili', 'guozhili', 'starpointcomm.com']
['=?utf-8?b?runjvc3lvkdnu6flrr4=?=', 'zhangjibin', 'ecit.org.cn']


In [7]:
import yaml
with open("/home/christovis/InternetGov/bigbang/tests/data/address_header_test_file.yaml", "r") as stream:
    try:
        emails = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [24]:
import bigbang.domain as domain

def get_froms(data):
    """
    Returns a dataframe with a row for every message of the archive, containing
    column entries for:
     - The From field of the email
     - The email address extracted from the From field
     - The domain of the From field
    This dataframe is computed the first time this method is called and then cached.
    Returns
    ----------
    froms: pandas.DataFrame
    """
    emails = data.apply(domain.extract_email)
    domains = data.apply(domain.extract_domain)
    froms = pd.concat(
        [data, emails, domains],
        axis=1,
        keys=["From", "email", "domain"],
    )
    return froms

froms = get_froms(pd.Series(list(emails.keys())))

In [ ]:
## mlist_name = "3GPP_TSG_SA_WG6"
mlist = ListservList.from_mbox(
    name=mlist_name,
    filepath=f"/home/christovis/InternetGov/bigbang-archives/3GPP/{mlist_name}.mbox",
    include_body=True,
)

In [30]:
mlist.period_of_activity()

[datetime.datetime(2016, 3, 14, 10, 7, 22, tzinfo=datetime.timezone.utc),
 datetime.datetime(2017, 2, 27, 8, 4, 27, tzinfo=datetime.timezone.utc)]

In [31]:
mlist.get_domainscount(header_fields=['from'])

{'from': 26}

In [5]:
mlist.df[["from", "comments-to"]].dropna()

,from,comments-to
0,tomas =?utf-8?q?toftg=c3=a5rd?= <tomas.toftgar...,milan.jelinek@usherbrooke.ca <milan.jelinek@us...
1,milan jelinek <milan.jelinek@usherbrooke.ca>,tomas =?utf-8?q?toftg=c3=a5rd?= <tomas.toftgar...
2,tomas =?utf-8?q?toftg=c3=a5rd?= <tomas.toftgar...,imre varga <ivarga@qti.qualcomm.com>
3,imre varga <ivarga@qti.qualcomm.com>,milan.jelinek@usherbrooke.ca <milan.jelinek@us...
4,milan jelinek <milan.jelinek@usherbrooke.ca>,imre varga <ivarga@qti.qualcomm.com>
5,imre varga <ivarga@qti.qualcomm.com>,milan.jelinek@usherbrooke.ca <milan.jelinek@us...
12,szczerba marek <marek.szczerba@philips.com>,imre varga <ivarga@qti.qualcomm.com>cc: bont f...
16,huan-yu su <hs@qosound.com>,imre varga <ivarga@qti.qualcomm.com>
17,multrus markus <markus.multrus@iis.fraunhofer.de>,imre varga <ivarga@qti.qualcomm.com>
18,stefan =?utf-8?q?d=c3=b6hla?= <stefan.doehla@i...,imre varga <ivarga@qti.qualcomm.com>


In [4]:
file_auth = CONFIG.config_path + "authentication.yaml"

mlist = ListservList.from_mbox(
    name="3GPP_TSG_SA_WG4_EVS",
    filepath=CONFIG.test_data_path + "3GPP_mbox/3GPP_TSG_SA_WG4_EVS.mbox",
)

march = ListservArchive.from_mbox(
    name="3GPP",
    directorypath=CONFIG.test_data_path + "3GPP_mbox/",
    filedsc="3GPP_TSG_*",
)

In [29]:
mlist.df[['from']].values

array([['tomas =?utf-8?q?toftg=c3=a5rd?= <tomas.toftgard@ericsson.com>'],
       ['milan jelinek <milan.jelinek@usherbrooke.ca>'],
       ['tomas =?utf-8?q?toftg=c3=a5rd?= <tomas.toftgard@ericsson.com>'],
       ['imre varga <ivarga@qti.qualcomm.com>'],
       ['milan jelinek <milan.jelinek@usherbrooke.ca>'],
       ['imre varga <ivarga@qti.qualcomm.com>'],
       ['milan jelinek <milan.jelinek@usherbrooke.ca>'],
       ['huan-yu su <hs@qosound.com>'],
       ['multrus markus <markus.multrus@iis.fraunhofer.de>'],
       ['bruhn stefan <stefan.bruhn@dolby.com>'],
       ['huan-yu su <hs@qosound.com>'],
       ['bruhn stefan <stefan.bruhn@dolby.com>'],
       ['szczerba marek <marek.szczerba@philips.com>'],
       ['imre varga <ivarga@qti.qualcomm.com>'],
       ['bruhn stefan <stefan.bruhn@dolby.com>'],
       ['imre varga <ivarga@qti.qualcomm.com>'],
       ['huan-yu su <hs@qosound.com>'],
       ['multrus markus <markus.multrus@iis.fraunhofer.de>'],
       ['stefan =?utf-8?q?d=c3=b6hl

In [33]:
mlist.crop_by_subject(match="EVS SWG Sessions", place=0).df['subject'].values

array(['Re: Information related to EVS SWG Sessions during SA4#115e meeting',
       'Information related to EVS SWG Sessions during SA4#115e meeting',
       'Information related to EVS SWG Sessions during SA4#114e meeting'],
      dtype=object)